## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
import numpy as np
import tensorflow as tf
from keras import backend as K

from keras.models import Model
from keras.models import Sequential

from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import GlobalAveragePooling2D
from keras.layers import Input

from keras.applications.xception import Xception
from keras.applications.resnet import ResNet50

from keras.datasets import cifar10
from sklearn.preprocessing import OneHotEncoder

from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [2]:
input_tensor = Input(shape=(32, 32, 3))

In [3]:
# Xception 架構
model_Xception = Xception(include_top=False, weights='imagenet',
                          input_tensor=input_tensor,
                          pooling=None, classes=10)
model_Xception.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [4]:
# Resnet 50 架構
model_Resnet50 = ResNet50(include_top=False, weights='imagenet',
                          input_tensor=input_tensor,
                          pooling=None, classes=10)
model_Resnet50.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

## 準備 Cifar 10 資料

In [5]:

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

## 決定model種類

In [6]:
model = model_Xception

## 添加層數

In [7]:
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.1)(x)

predictions = Dense(10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions,name=model.name.capitalize())

print(f'Model name : {model.name.capitalize()}\nModel deepth :{len(model.layers)}')

Model name : Xception
Model deepth :136


## 鎖定特定幾層不要更新權重

In [8]:
lock_num = 100
for i, layer in enumerate(model.layers):
    layer.trainable = False if i < lock_num else True
print(f'Lock top {lock_num} layers.')  

Lock top 100 layers.


## Training

In [9]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
callbacks = [ReduceLROnPlateau(monitor="val_accuracy", factor=0.5, patience=3, verbose=1, min_lr=0.0001),
              EarlyStopping(monitor = 'val_loss', patience=5, verbose=1)]

history = model.fit(x_train, y_train, validation_split=0.2, batch_size=128, epochs=100, callbacks=callbacks)

Epoch 1/100
313/313 [==============================] - 80s 94ms/step - loss: 1.7067 - accuracy: 0.3844 - val_loss: 0.9999 - val_accuracy: 0.6650
Epoch 2/100
313/313 [==============================] - 24s 77ms/step - loss: 0.8338 - accuracy: 0.7140 - val_loss: 0.9033 - val_accuracy: 0.6854
Epoch 3/100
313/313 [==============================] - 25s 81ms/step - loss: 0.6161 - accuracy: 0.7861 - val_loss: 0.8767 - val_accuracy: 0.7099
Epoch 4/100
313/313 [==============================] - 26s 83ms/step - loss: 0.4438 - accuracy: 0.8486 - val_loss: 0.9347 - val_accuracy: 0.7222
Epoch 5/100
313/313 [==============================] - 26s 83ms/step - loss: 0.3156 - accuracy: 0.8945 - val_loss: 0.9946 - val_accuracy: 0.7182
Epoch 6/100
313/313 [==============================] - 25s 80ms/step - loss: 0.2156 - accuracy: 0.9273 - val_loss: 1.0869 - val_accuracy: 0.7188
Epoch 7/100
313/313 [==============================] - 26s 82ms/step - loss: 0.1580 - accuracy: 0.9496 - val_loss: 1.2288 - val_ac

In [10]:
# 不鎖定的話
model = model_Xception

x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.1)(x)

predictions = Dense(10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions,name=model.name.capitalize())
print(f'Model name : {model.name.capitalize()}\nModel deepth :{len(model.layers)}')

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
callbacks = [ReduceLROnPlateau(monitor="val_accuracy", factor=0.5, patience=3, verbose=1, min_lr=0.0001),
              EarlyStopping(monitor = 'val_loss', patience=5, verbose=1)]

history = model.fit(x_train, y_train, validation_split=0.2, batch_size=128, epochs=100, callbacks=callbacks)

Model name : Xception
Model deepth :136
Epoch 1/100
313/313 [==============================] - 45s 95ms/step - loss: 0.2682 - accuracy: 0.9397 - val_loss: 1.3762 - val_accuracy: 0.7129
Epoch 2/100
313/313 [==============================] - 27s 85ms/step - loss: 0.1017 - accuracy: 0.9678 - val_loss: 1.3545 - val_accuracy: 0.7152
Epoch 3/100
313/313 [==============================] - 26s 83ms/step - loss: 0.0855 - accuracy: 0.9727 - val_loss: 1.2975 - val_accuracy: 0.7185
Epoch 4/100
313/313 [==============================] - 28s 89ms/step - loss: 0.0735 - accuracy: 0.9765 - val_loss: 1.4074 - val_accuracy: 0.7208
Epoch 5/100
313/313 [==============================] - 27s 87ms/step - loss: 0.0674 - accuracy: 0.9775 - val_loss: 1.4248 - val_accuracy: 0.7193
Epoch 6/100
313/313 [==============================] - 28s 89ms/step - loss: 0.0629 - accuracy: 0.9794 - val_loss: 1.5146 - val_accuracy: 0.7215
Epoch 7/100
313/313 [==============================] - 28s 90ms/step - loss: 0.0525 - accu

In [11]:
# 換一種試試
model = model_Resnet50

x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.1)(x)

predictions = Dense(10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions,name=model.name.capitalize())
print(f'Model name : {model.name.capitalize()}\nModel deepth :{len(model.layers)}')

lock_num = 100
for i, layer in enumerate(model.layers):
    layer.trainable = False if i < lock_num else True
print(f'Lock top {lock_num} layers.')    

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
callbacks = [ReduceLROnPlateau(monitor="val_accuracy", factor=0.5, patience=3, verbose=1, min_lr=0.0001),
              EarlyStopping(monitor = 'val_loss', patience=5, verbose=1)]

history = model.fit(x_train, y_train, validation_split=0.2, batch_size=128, epochs=100, callbacks=callbacks)

Model name : Resnet50
Model deepth :179
Lock top 100 layers.
Epoch 1/100
313/313 [==============================] - 72s 148ms/step - loss: 1.9523 - accuracy: 0.3370 - val_loss: 2.1472 - val_accuracy: 0.3219
Epoch 2/100
313/313 [==============================] - 36s 114ms/step - loss: 1.4470 - accuracy: 0.4901 - val_loss: 1.7665 - val_accuracy: 0.4127
Epoch 3/100
313/313 [==============================] - 37s 117ms/step - loss: 1.2724 - accuracy: 0.5564 - val_loss: 1.9710 - val_accuracy: 0.4216
Epoch 4/100
313/313 [==============================] - 36s 115ms/step - loss: 1.1470 - accuracy: 0.5976 - val_loss: 1.8310 - val_accuracy: 0.4269
Epoch 5/100
313/313 [==============================] - 36s 114ms/step - loss: 1.0465 - accuracy: 0.6293 - val_loss: 13.6338 - val_accuracy: 0.1233
Epoch 6/100
313/313 [==============================] - 39s 125ms/step - loss: 1.0022 - accuracy: 0.6440 - val_loss: 1.6103 - val_accuracy: 0.4597
Epoch 7/100
313/313 [==============================] - 37s 120

In [12]:
model = model_Resnet50

x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.1)(x)

predictions = Dense(10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions,name=model.name.capitalize())
print(f'Model name : {model.name.capitalize()}\nModel deepth :{len(model.layers)}')

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
callbacks = [ReduceLROnPlateau(monitor="val_accuracy", factor=0.5, patience=3, verbose=1, min_lr=0.0001),
              EarlyStopping(monitor = 'val_loss', patience=5, verbose=1)]

history = model.fit(x_train, y_train, validation_split=0.2, batch_size=128, epochs=100, callbacks=callbacks)

Model name : Resnet50
Model deepth :179
Epoch 1/100
313/313 [==============================] - 64s 133ms/step - loss: 1.1689 - accuracy: 0.6405 - val_loss: 2.6272 - val_accuracy: 0.3890
Epoch 2/100
313/313 [==============================] - 37s 118ms/step - loss: 0.5156 - accuracy: 0.8227 - val_loss: 3.8008 - val_accuracy: 0.3687
Epoch 3/100
313/313 [==============================] - 38s 120ms/step - loss: 0.3762 - accuracy: 0.8729 - val_loss: 5.2485 - val_accuracy: 0.2837
Epoch 4/100
313/313 [==============================] - 38s 121ms/step - loss: 0.3384 - accuracy: 0.8858 - val_loss: 6.3028 - val_accuracy: 0.2675

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 5/100
313/313 [==============================] - 36s 117ms/step - loss: 0.1846 - accuracy: 0.9411 - val_loss: 2.8125 - val_accuracy: 0.4773
Epoch 6/100
313/313 [==============================] - 38s 121ms/step - loss: 0.1057 - accuracy: 0.9682 - val_loss: 2.3415 - val_accuracy: 0.5555
Epo